In [128]:
import pandas as pd

In [129]:
import numpy as np 
import random

In [130]:
import pulp as p

In [131]:
player = [str(i) for i in range(200)]

In [132]:
teams= ['A','B','C','D']

In [133]:
p_team = [random.choice(teams) for i in range(200)  ]

In [134]:
pos =['QB','RB','WR','TE','K','TD']

In [135]:
p_pos = [ random.choice(pos) for i in range(200)]

In [136]:

for j in teams:
    c=[]
    for i in range(200):
        if p_team[i] == j :
            c.append(p_pos[i])
    print(np.unique(np.array(c)) )
        

['K' 'QB' 'RB' 'TD' 'TE' 'WR']
['K' 'QB' 'RB' 'TD' 'TE' 'WR']
['K' 'QB' 'RB' 'TD' 'TE' 'WR']
['K' 'QB' 'RB' 'TD' 'TE' 'WR']


In [137]:
opp_team=[]
for i in teams:
    c=teams.copy()
    c.remove(i)
    opp_team.append(c)


In [138]:
opp_team[1][1]='D'
opp_team[1][2]='C'
opp_team[2][0]='D'
opp_team[2][1]='A'
opp_team[2][2]='B'
opp_team[3][0]='C'
opp_team[3][1]='B'
opp_team[3][2]='A'

In [139]:
opp_team

[['B', 'C', 'D'], ['A', 'D', 'C'], ['D', 'A', 'B'], ['C', 'B', 'A']]

In [140]:
data=[]
data.append(player)
data.append(p_team)
data.append(p_pos)
data = list(np.transpose(np.array(data)) )

In [141]:
df = pd.DataFrame(data=data ,columns = ['Player','team','pos'])

In [142]:
df_opp = pd.DataFrame(data = opp_team,columns=['week1','week2','week3'])

In [143]:
df_opp['team'] = teams

In [144]:
df_opp

,week1,week2,week3,team
0,B,C,D,A
1,A,D,C,B
2,D,A,B,C
3,C,B,A,D


In [145]:
week1 = [random.randint(1,40) for i in range(200)   ]
week2 = [random.randint(1,40) for i in range(200)   ]
week3 = [random.randint(1,40) for i in range(200)   ]


In [146]:
df['fpweek1'] = week1
df['fpweek2'] = week2
df['fpweek3'] = week3

# Created a random Dataset 

In [147]:
df.head()

,Player,team,pos,fpweek1,fpweek2,fpweek3
0,0,B,TD,31,4,12
1,1,A,K,20,36,7
2,2,A,RB,23,34,23
3,3,C,QB,33,14,9
4,4,D,RB,9,16,31


In [148]:
r= random.sample(range(200),200)

In [149]:
df['rank'] = r

# Assigning Ranks to each player

In [150]:
ls=[]
for i in range(200):
    s = df['fpweek1'][i] + df['fpweek2'][i] +df['fpweek3'][i]
    ls.append(s)
s =sorted(ls,reverse=True)
    

In [151]:
r=[]
for i in range(200):
    x = df['fpweek1'][i] + df['fpweek2'][i] +df['fpweek3'][i]
    ind = s.index(x)
    r.append(ind)
df['rank']=r
    

# Giving the set of possible Fantasy Points Required for each week

In [152]:
fpreq = [random.randint(260,350) for i in range(3)]

In [153]:
fpreq

[281, 321, 310]

# Excluding the ranks chosen by our opponents

In [154]:
def opp_ranks(rank):
    r = rank
    total = 4
    ranks =[]
    prev =r
    for i in range(1,200):
        
        if i%2==0:
            r = r+total+1
        else:
            r = r+total-1
        
        num = random.randint(1,3)
        selects = [random.randint(prev,r) for j in range(num)]
        prev = r
        for j in selects:
            ranks.append(j)
            
                    
    final_r =[1]*200
    for i in range(200):
        if df['rank'][i] in ranks:
            final_r[i]=0
    return final_r
    
    
    

# Mapping each player with their position

In [155]:
qb=[]
rb=[]
wr=[]
te=[]
k=[]
td=[]

for i in df['pos']:
    if i=='QB':
        qb.append(1)
    else :
        qb.append(0)
    if i=='RB':
        rb.append(1)
    else :
        rb.append(0)
    if i=='WR':
        wr.append(1)
    else :
        wr.append(0)
    if i=='TE':
        te.append(1)
    else :
        te.append(0)
    if i=='K':
        k.append(1)
    else :
        k.append(0)
    if i=='TD':
        td.append(1)
    else :
        td.append(0)



# Integer Programming

In [156]:
prob = p.LpProblem('Problem',p.LpMaximize)

#The order given in draft phase
pos = random.randint(1,4)

#Mapping the required ranks from which we can chose players
ranks = opp_ranks(pos)

num_player =200
selected = p.LpVariable.dict('select',range(num_player),cat='Binary')
#Variable for week 1
x1 = p.LpVariable.dict('x1',range(num_player),cat='Binary')
#Variable for week 2
x2 = p.LpVariable.dict('x2',range(num_player),cat='Binary')
#Variable for week 3
x3 = p.LpVariable.dict('x3',range(num_player),cat='Binary')


# Objective Function to maximize Fantasy points
prob += sum([x1[i]*df['fpweek1'][i] for i in range(num_player)]) + sum([x2[i]*df['fpweek2'][i] for i in range(num_player)]) + sum([x3[i]*df['fpweek3'][i] for i in range(num_player)]) 


#Contraints

prob+= sum(selected[i] for i in range(num_player))<=50


# The number of players required each week
#forx1
prob+= sum([x1[i] for i in range(num_player)]) ==9

prob += sum([x1[i]*qb[i] for i in range(num_player)] )==1

prob += sum([x1[i]*rb[i] for i in range(num_player) ])==2

prob += sum([x1[i]*wr[i] for i in range(num_player)] )==3

prob += sum([x1[i]*te[i] for i in range(num_player)] )==1

prob += sum([x1[i]*k[i] for i in range(num_player)] )==1

prob += sum([x1[i]*td[i] for i in range(num_player)] )==1

#forx2
prob+= sum([x2[i] for i in range(num_player)]) ==9

prob += sum([x2[i]*qb[i] for i in range(num_player)] )==1

prob += sum([x2[i]*rb[i] for i in range(num_player) ])==2

prob += sum([x2[i]*wr[i] for i in range(num_player)] )==3

prob += sum([x2[i]*te[i] for i in range(num_player)] )==1

prob += sum([x2[i]*k[i] for i in range(num_player)] )==1

prob += sum([x2[i]*td[i] for i in range(num_player)] )==1

#forx3
prob+= sum([x3[i] for i in range(num_player)]) ==9

prob += sum([x3[i]*qb[i] for i in range(num_player)] )==1

prob += sum([x3[i]*rb[i] for i in range(num_player) ])==2

prob += sum([x3[i]*wr[i] for i in range(num_player)] )==3

prob += sum([x3[i]*te[i] for i in range(num_player)] )==1

prob += sum([x3[i]*k[i] for i in range(num_player)] )==1

prob += sum([x3[i]*td[i] for i in range(num_player)] )==1

#The Fantasy points for beating the opponent
#for winning
prob+= sum([x1[i]*df['fpweek1'][i] for i in range(num_player)])>=fpreq[0]
prob+= sum([x2[i]*df['fpweek2'][i] for i in range(num_player)])>=fpreq[1]
prob+= sum([x3[i]*df['fpweek3'][i] for i in range(num_player)])>=fpreq[2]
prob+= sum([selected[i]*df['fpweek1'][i] for i in range(num_player)])>=fpreq[0]
prob+= sum([selected[i]*df['fpweek2'][i] for i in range(num_player)])>=fpreq[1]
prob+= sum([selected[i]*df['fpweek3'][i] for i in range(num_player)])>=fpreq[2]


#The ranks from which players can be chosen

prob+= sum([x1[i]*ranks[i] for i in range(num_player)])==9
prob+= sum([x2[i]*ranks[i] for i in range(num_player)])==9
prob+= sum([x3[i]*ranks[i] for i in range(num_player)])==9
prob += sum([selected[i]*ranks[i] for i in range(num_player)] )<=50




In [157]:
prob.solve()

1

In [158]:
sel=[]
for i in range(200):
    if p.value(x1[i])>=0.9 and p.value(x1[i])<=1.1:
        sel.append(1)
    if p.value(x2[i])>=0.9 and p.value(x2[i])<=1.1:
        sel.append(1)
    if p.value(x3[i])>=0.9 and p.value(x3[i])<=1.1:
        sel.append(1)

    else:
        sel.append(0)

# Final List of Players Chosen

In [159]:
total = 0
final=[]
for i in range(200):
    if sel[i]==1:
        print([df['Player'][i], df['pos'][i] , df['fpweek1'][i] , df['fpweek2'][i],df['fpweek3'][i] ])
        final.append([df['Player'][i], df['pos'][i] , df['fpweek1'][i] , df['fpweek2'][i],df['fpweek3'][i] ])
        total+=  df['fpweek1'][i] +  df['fpweek2'][i]+ df['fpweek3'][i]
print(total)


['5', 'WR', 34, 6, 38]
['9', 'QB', 1, 29, 40]
['21', 'WR', 38, 39, 3]
['22', 'WR', 35, 23, 22]
['41', 'RB', 11, 40, 25]
['55', 'WR', 26, 20, 2]
['68', 'K', 14, 27, 32]
['74', 'WR', 7, 13, 34]
['82', 'TD', 11, 32, 1]
['97', 'RB', 12, 13, 8]
['111', 'RB', 10, 6, 33]
['126', 'QB', 7, 6, 7]
['138', 'QB', 38, 6, 25]
['140', 'TE', 20, 11, 7]
['145', 'K', 14, 27, 6]
['153', 'TD', 21, 9, 27]
['154', 'QB', 8, 15, 12]
['162', 'TD', 33, 35, 32]
['172', 'RB', 36, 36, 3]
['176', 'K', 23, 28, 7]
['182', 'TE', 37, 40, 20]
['192', 'TE', 18, 31, 23]
['195', 'QB', 12, 25, 17]
['196', 'WR', 31, 5, 18]
['197', 'K', 16, 6, 5]
1488
